In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_virtual_device_configuration(
                gpu,
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]  # Limit to 4GB
            )
        print("GPU memory limit set")
    except RuntimeError as e:
        print(e)


GPU memory limit set


In [2]:
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy('mixed_float16')


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce GTX 1650 Ti, compute capability 7.5


In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten 
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input

import zipfile

In [4]:
path = r'D:\ML projects\gender_age_emotion_dataset\utkface_aligned_cropped\UTKFace'

images = []
age = []
gender = []


for image in  os.listdir(path):
    ages = image.split('_')[0]
    genders = image.split('_')[1] 
    image = cv2.imread(str(path) + '/' + str(image))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    age.append(np.array(ages))
    gender.append(np.array(genders))
    images.append(np.array(image))
    




In [5]:
age = np.array(age, dtype = np.int64)
gender = np.array(gender, dtype = np.uint64)
images = np.array(images)

In [6]:
len(age), len(gender), len(images)

(23708, 23708, 23708)

In [7]:
from sklearn.model_selection import train_test_split
X_train_age, X_test_age, y_train_age, y_test_age = train_test_split(images, age, test_size = 0.2, random_state = 42)
X_train_gender, X_test_gender, y_train_gender, y_test_gender = train_test_split(images, gender, test_size = 0.2, random_state = 42)

In [8]:
from tensorflow.keras.applications import ResNet50

pre_trained_model = ResNet50(weights = 'imagenet',include_top = False )


for layer in pre_trained_model.layers:
    layer.trainable = False

In [9]:
age_model = Sequential()

age_model.add(Input(shape = (200, 200, 3)))
age_model.add(pre_trained_model)




age_model.add(Conv2D(32,(3,3), activation = 'relu', padding = 'same' ))
age_model.add(MaxPooling2D(2,2))

age_model.add(Conv2D(64,(3,3), activation = 'relu' , padding = 'same'))
age_model.add(MaxPooling2D(2,2))

age_model.add(Flatten())

age_model.add(Dense(64, activation = 'relu'))
age_model.add(Dropout(0.2))



age_model.add(Dense(1, activation = 'linear'))


In [10]:
gender_model = Sequential()

gender_model.add(Input(shape = (200, 200, 3)))
gender_model.add(pre_trained_model)



gender_model.add(Conv2D(32,(3,3), activation = 'relu', padding = 'same' ))
gender_model.add(MaxPooling2D(2,2))

gender_model.add(Conv2D(64,(3,3), activation = 'relu' , padding = 'same'))
gender_model.add(MaxPooling2D(2,2))

gender_model.add(Flatten())


gender_model.add(Dense(32, activation = 'relu'))
gender_model.add(Dropout(0.2))




gender_model.add(Dense(1, activation = 'sigmoid'))

In [11]:
age_model.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])

In [14]:
history_age = age_model.fit(X_train_age, y_train_age, validation_data = (X_test_age, y_test_age),  epochs = 10)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
age_model.save('age_model.h5')

In [ ]:
from tensorflow.keras.models import load_model 
age_model = load_model('age_model.h5')

In [ ]:
gender_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history_gender = gender_model.fit(X_train_gender, y_train_gender, validation_data = (X_test_gender, y_test_gender),  epochs = 10)

In [ ]:
with tf.device('/CPU:0'):
    history_gender = gender_model.fit(X_train_gender, y_train_gender, validation_data=(X_test_gender, y_test_gender), epochs=10)


In [ ]:
gender_model.save('gender_model.h5')

In [ ]:
import h5py

try:
    with h5py.File("age_model.h5", "r") as f:
        print("File is readable!")
except OSError:
    print("The file is corrupted.")
